Here we will create a simulation in two ways. First, we will load solar system bodies from NASA's HORIZONS database. Then, we will create a system from scratch by adding a star and then one planet at a time. Once we have saved a simulation, we can then calculate the musical pitches that correspond to each orbiting body.

# Adding particles using NASA JPL Horizons Database
Solar system bodies in the Horizons database can be added to a simulation using either their IAU name or their NAIF-ID number (https://naif.jpl.nasa.gov/pub/naif/toolkit_docs/MATLAB/req/naif_ids.html). As an example we will load Saturn and its major moons on June 4, 2017 at 5:00 UTC. If no date is specified, rebound will use the current date (when the first particle is added). Either way, this date is cached and used for all other objects that are added to the simulation. Note that running this cell this will overwrite any simulation with the same name!

For more information on using the Horizons database with REBOUND see: http://rebound.readthedocs.io/en/latest/ipython/Horizons.html

In [2]:
import sys
sys.path.append('../')
import rebound
filename = "test.bin"

date = "2017-06-04 05:00"
sim = rebound.Simulation()
sim.add("Saturn", date=date)
sim.add("Mimas")
sim.add("Enceladus")
sim.add("Tethys")
sim.add("Dione")
sim.add("Rhea")
sim.add("Titan")
sim.add("Iapetus")
    
sim.move_to_com()
sim.save("../binaries/"+filename)


Searching NASA Horizons for 'Saturn'... Found: Saturn Barycenter (6).
Searching NASA Horizons for 'Mimas'... Found: Mimas (601).
Searching NASA Horizons for 'Enceladus'... Found: Enceladus (602).
Searching NASA Horizons for 'Tethys'... Found: Tethys (603).
Searching NASA Horizons for 'Dione'... Found: Dione (604).
Searching NASA Horizons for 'Rhea'... Found: Rhea (605).
Searching NASA Horizons for 'Titan'... Found: Titan (606).
Searching NASA Horizons for 'Iapetus'... Found: Iapetus (608).


# Creating A System From Scratch
You can also create a new planetary system with arbitrary masses and orbital parameters. As an example, we will add a star with the mass of the sun and then 3 low mass planets at the right semi-major axes so that their orbital frequencies will create a major chord. We will place the planets on circular, non-inclined orbits but at random longitudes.

More information on orbital elements in REBOUND can be found here: http://rebound.readthedocs.io/en/latest/ipython/OrbitalElements.html

In [7]:
import sys
sys.path.append('../')
import systemsounds as ss
import numpy as np
import rebound
import random
filename = "majorchord.bin"

sim = rebound.Simulation()
sim.add(m=1.)  #add 2 Solar mass star at the origin

m_planet=1e-6     #planet mass, in units of solar masses 
a1=1.             #semi-major axis of planet 1 in AU
a2=a1*(5./4.)**(-2./3)  #semi-major axis of a major 3rd above planet 1  (5:4 resonance)
a3=a1*(3./2.)**(-2./3)  #semi-major axis of a perfect 5th above planet 1 (3:2 resonance)

#add the 3 planets, ordered from smallest to largest semi-major axis
#you can also set non-zero eccentricty, inclination and other orbital parameters 
sim.add(m=m_planet, a=a3, e=0., inc=0., theta=2.*np.pi*np.random.rand()) 
sim.add(m=m_planet, a=a2, theta=2.*np.pi*np.random.rand())
sim.add(m=m_planet, a=a1, theta=2.*np.pi*np.random.rand())
 
sim.move_to_com()
sim.save("../binaries/"+filename)

# Calculating The Pitches Of A Planetary System
Once a simulation has been created, we can load it here and calculate the musical pitches associated with each planet. In WritingMIDIfile.ipynb, this algorithm is used to assign pitches to planets but the notes must be rounded to standard pitches to be expressed in MIDI. Here, we will see what the exact pitches would be without rounding. You may want to recever these frequencies by loading the MIDI file into a DAW (Garageband, Logic, ProTools), splitting the midi file into separate tracks for each note, and then using a pitch shifter plugin.

We calculate the pithes of our system by choosing an arbitray base frequency for one of the planets and then calulating the pitches of the other planets relative to it. If you know the actual orbital period of one of the objects (this is likely different from the orbitla period within rebound), you can instead specify the number of octaves you'd like to increase the pitch by. As an example, we'll load majorchord.sim and scale the frequencies so that the outer planet corresponds to C4. You may notice that planets with exact whole number period ratios do not correspond to exact standard pitches due to the discrepency between 'Just' and 'Even' temperment.

In [17]:
notenames = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
f_A4 = 440.
f_C0 = f_A4*pow(2, -4.75014)

def pitch(freq):
    h = int(round(12*np.log2(freq/f_C0)))
    octave = h // 12
    n = h % 12
    f0=f_C0*2**(h/12.)
    cent = 1200*np.log2(freq/f0)
    return '{0} {1:+.1f} cents'.format(notenames[n]+str(octave),cent)

filename = "../binaries/majorchord.bin"
sim = rebound.Simulation.from_file(filename)

#set outermost planet to the frequency of C4 (or any other frequncy in Hz)
f_C4=f_C0*pow(2, 4)
basefreq=f_C4

##OR: scale up orbital frequencies by a set number of octaves
#Pouter=1.*3.154e7 #1 year in seconds (orbital oeriod of outermost planet)
#octaves=22
#basefreq=pow(2.,octaves)/Pouter

baseperiod=sim.particles[-1].P
freqs=[basefreq*baseperiod/sim.particles[i].P for i in range(1,sim.N)]
print('\nNotes and offsets:')
for i in range(1,sim.N):
    print('Planet '+str(i)+' : '+ pitch(freqs[i-1]),' = ',np.round(freqs[i-1],2), 'Hz')


Notes and offsets:
Planet 1 : G4 +2.0 cents  =  392.4 Hz
Planet 2 : E4 -13.7 cents  =  327.0 Hz
Planet 3 : C4 +0.0 cents  =  261.6 Hz
